## Importar librerías

In [2]:
# Importar librerías
import numpy as np
import pandas as pd

# Importar librerías de procesamiento de texto
import re, os
import nltk
import json
from nltk.corpus import stopwords
from wordcloud import WordCloud

# Importar librerías de modelado y entrenamiento
import pycaret
import transformers
from transformers import AutoModel, BertTokenizerFast, AdamW
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from sklearn.metrics import plot_confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

# Importar librerías de PyTorch para procesamiento de datos y modelos
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch
import torch.nn as nn




## Funciones para datos iniciales

In [3]:
def remove_nan_rows(dataframe, columns=['messages', 'sender_labels', 'receiver_labels']):
    """
    Elimina las filas con valores NaN en columnas especificadas.

    Parámetros:
    - dataframe: pd.DataFrame
        El DataFrame a procesar.
    - columns: list
        Lista de columnas en las que buscar valores NaN.

    Retorna:
    - pd.DataFrame
        DataFrame con las filas que contienen valores NaN eliminadas.
    """
    return dataframe.dropna(subset=columns)


def remove_tags(string, remove_special_chars=False, remove_stopwords=False, remove_newlines=False):
    """
    Elimina etiquetas y contenido no deseado de una cadena de texto, como etiquetas HTML, nombres de usuario de Twitter, hashtags, números y URLs.

    Parámetros:
    - string: str
        Cadena de texto a procesar.
    - remove_special_chars: bool
        Indica si se deben eliminar caracteres no alfanuméricos.
    - remove_stopwords: bool
        Indica si se deben eliminar palabras comunes (stopwords).
    - remove_newlines: bool
        Indica si se deben eliminar caracteres de nueva línea.

    Retorna:
    - str
        Cadena de texto procesada y limpia.
    """
    result = re.sub(r'<.*?>', '', string)  # Elimina etiquetas HTML
    result = re.sub('@[\w]+', '', result)  # Elimina nombres de usuario de Twitter
    result = re.sub('#[\w]+', '', result)  # Elimina hashtags
    result = re.sub("\d+", " ", result)  # Elimina números
    result = re.sub(r'http\S+', '', result)  # Elimina URLs

    if remove_special_chars:
        result = re.sub(r'[^\w\s]', ' ', result)  # Elimina caracteres no alfanuméricos

    if remove_newlines:
        result = re.sub(r'\n\n', ' ', result)  # Elimina caracteres de nueva línea
        result = ' '.join(result.split())  # Divide y une para eliminar espacios adicionales

    if remove_stopwords:
        stop_words = set(stopwords.words('english'))
        result = ' '.join([w for w in result.split() if w.lower() not in stop_words])

    # Elimina palabras de longitud 1
    result = ' '.join([word for word in result.split() if len(word) > 1])

    result = result.lower()
    return result


def compute_class_weight(train_y):
    """
    Calcula el peso de las clases dado un conjunto de datos de entrenamiento desbalanceado.
    Generalmente utilizado en modelos de redes neuronales para ajustar la función de pérdida (función de pérdida ponderada),
    dando más peso a las clases raras.

    Parámetros:
    - train_y: array
        Etiquetas del conjunto de datos de entrenamiento.

    Retorna:
    - dict
        Diccionario que asigna el peso a cada clase.
    """
    import sklearn.utils.class_weight as scikit_class_weight

    class_list = list(set(train_y))
    class_weight_value = scikit_class_weight.compute_class_weight(class_weight='balanced', classes=class_list, y=train_y)
    class_weight = dict()

    # Inicializa todas las clases en el diccionario con peso 1
    curr_max = int(np.max(class_list))
    for i in range(curr_max):
        class_weight[i] = 1

    # Construye el diccionario utilizando el peso obtenido de la función de scikit
    for i in range(len(class_list)):
        class_weight[class_list[i]] = class_weight_value[i]

    return class_weight


## Importar y Preparar Datos

In [4]:
# Paso 1: Cargar datos de entrenamiento desde el archivo 'train.jsonl'
data_list = []
messages = []
with open('NLP_Diplomacy/train.jsonl', 'r') as archivo:
    for line in archivo:
        try:
            data = json.loads(line)
            data_list.append(data)
            messages.extend(data['messages'])
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON: {e}")

# Paso 2: Cargar datos de validación desde el archivo 'validation.jsonl'
validation_list = []
messages = []
with open('NLP_Diplomacy/validation.jsonl', 'r') as archivo:
    for line in archivo:
        try:
            validation = json.loads(line)
            validation_list.append(validation)
            messages.extend(validation['messages'])
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON: {e}")

# Paso 3: Cargar datos de prueba desde el archivo 'test.jsonl'
test_list = []
messages = []
with open('NLP_Diplomacy/test.jsonl', 'r') as archivo:
    for line in archivo:
        try:
            test = json.loads(line)
            test_list.append(validation)
            messages.extend(validation['messages'])
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON: {e}")

# Paso 4: Crear DataFrames a partir de las listas de datos
df = pd.DataFrame(data_list)
df_val = pd.DataFrame(validation_list)
df_test = pd.DataFrame(test_list)

# Paso 5: "Explotar" las columnas de listas en filas individuales
df_explode = df.explode(['messages', 'sender_labels', 'receiver_labels', 'speakers', 'receivers', 'absolute_message_index', 'relative_message_index', 'seasons', 'years', 'game_score', 'game_score_delta'], ignore_index=True)
df_val = df_val.explode(['messages', 'sender_labels', 'receiver_labels', 'speakers', 'receivers', 'absolute_message_index', 'relative_message_index', 'seasons', 'years', 'game_score', 'game_score_delta'], ignore_index=True)
df_test = df_test.explode(['messages', 'sender_labels', 'receiver_labels', 'speakers', 'receivers', 'absolute_message_index', 'relative_message_index', 'seasons', 'years', 'game_score', 'game_score_delta'], ignore_index=True)

# Paso 6: Limpiar filas con valores NaN
df_explode_cleaned = remove_nan_rows(df_explode)
df_val_cleaned = remove_nan_rows(df_val)
df_test_cleaned = remove_nan_rows(df_test)

# Paso 7: Convertir la columna 'messages' a tipo de dato string y limpiar el texto
df_explode_cleaned['messages'] = df_explode_cleaned['messages'].astype(str)
df_explode_cleaned.loc[:, 'messages_clean'] = df_explode_cleaned['messages'].apply(lambda cw: remove_tags(cw, remove_special_chars=True, remove_stopwords=True, remove_newlines=True))

df_val_cleaned['messages'] = df_val_cleaned['messages'].astype(str)
df_val_cleaned.loc[:, 'messages_clean'] = df_val_cleaned['messages'].apply(lambda cw: remove_tags(cw, remove_special_chars=True, remove_stopwords=True, remove_newlines=True))

df_test_cleaned['messages'] = df_test_cleaned['messages'].astype(str)
df_test_cleaned.loc[:, 'messages_clean'] = df_test_cleaned['messages'].apply(lambda cw: remove_tags(cw, remove_special_chars=True, remove_stopwords=True, remove_newlines=True))

# Paso 8: Limpiar filas con valores NaN después de la limpieza adicional
df_explode_cleaned = remove_nan_rows(df_explode_cleaned)
df_val_cleaned = remove_nan_rows(df_val_cleaned)
df_test_cleaned = remove_nan_rows(df_test_cleaned)

# Paso 9: Calcular la longitud del texto en cada fila
df_explode_cleaned['caracteres_texto'] = df_explode_cleaned['messages_clean'].apply(lambda row: len(str(row)) if not pd.isna(row) else np.nan)

# Paso 10: Filtrar DataFrame con condiciones específicas
filtered_dfb = df_explode_cleaned[(df_explode_cleaned['sender_labels'] == False) & (df_explode_cleaned['receiver_labels'] == True) | 
                                  (df_explode_cleaned['sender_labels'] == True) & (df_explode_cleaned['receiver_labels'] == False)| 
                                  (df_explode_cleaned['sender_labels'] == False) & (df_explode_cleaned['receiver_labels'] == False)| 
                                  (df_explode_cleaned['sender_labels'] == True) & (df_explode_cleaned['receiver_labels'] == True) & (df_explode_cleaned['caracteres_texto'] <= 20)]


## Preparando Datos para el modelo

In [5]:
df_explode_cleaned['caracteres_texto'].median(), df_explode_cleaned['caracteres_texto'].min(), df_explode_cleaned['caracteres_texto'].max()

(41.0, 0, 878)

In [6]:
# Cargar modelo BERT y tokenizador a través de HuggingFace Transformers
bert = AutoModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

In [7]:
# Paso 11: Obtener mensajes de entrenamiento y etiquetas asociadas
train_messages = filtered_dfb['messages_clean'].values
train_labels = filtered_dfb["sender_labels"].values

# Paso 12: Obtener mensajes de validación y etiquetas asociadas
val_messages = df_val_cleaned['messages_clean'].values
val_labels = df_val_cleaned["sender_labels"].values

# Paso 13: Obtener mensajes de prueba y etiquetas asociadas
test_messages = df_test_cleaned['messages_clean'].values
test_labels = df_test_cleaned["sender_labels"].values

# Paso 14: Inicializar y ajustar el codificador de etiquetas
encoder = LabelEncoder()

# Paso 15: Codificar las etiquetas para entrenamiento, validación y prueba
train_encoded_labels = encoder.fit_transform(train_labels)
val_encoded_labels = encoder.fit_transform(val_labels)
test_encoded_labels = encoder.fit_transform(test_labels)

In [8]:
# La mayoría de mensajes tiene una media de palabras de 41. 
# Por lo tanto, establecemos la longitud máxima del título en 50.
MAX_LENGHT = 50

# Tokenizar y codificar secuencias en el conjunto de entrenamiento
tokens_train = tokenizer.batch_encode_plus(
    train_messages.tolist(),
    max_length=MAX_LENGHT,
    pad_to_max_length=True,
    truncation=True
)

# Tokenizar y codificar secuencias en el conjunto de validación
tokens_val = tokenizer.batch_encode_plus(
    val_messages.tolist(),
    max_length=MAX_LENGHT,
    pad_to_max_length=True,
    truncation=True
)

# Tokenizar y codificar secuencias en el conjunto de prueba
tokens_test = tokenizer.batch_encode_plus(
    test_messages.tolist(),
    max_length=MAX_LENGHT,
    pad_to_max_length=True,
    truncation=True
)

In [9]:
# Convertir listas a tensores
train_seq = torch.tensor(tokens_train['input_ids'])
train_mask = torch.tensor(tokens_train['attention_mask'])
train_y = torch.tensor(train_labels.tolist())

val_seq = torch.tensor(tokens_val['input_ids'])
val_mask = torch.tensor(tokens_val['attention_mask'])
val_y = torch.tensor(val_labels.tolist())

test_seq = torch.tensor(tokens_test['input_ids'])
test_mask = torch.tensor(tokens_test['attention_mask'])
test_y = torch.tensor(test_labels.tolist())

In [10]:
# Definición de la estructura del DataLoader

batch_size = 32                                               # definir el tamaño del lote

train_data = TensorDataset(train_seq, train_mask, train_y)    # envolver los tensores
train_sampler = RandomSampler(train_data)                     # muestreador para tomar muestras de los datos durante el entrenamiento
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
                                                              # DataLoader para el conjunto de entrenamiento

val_data = TensorDataset(val_seq, val_mask, val_y)            # envolver los tensores
val_sampler = SequentialSampler(val_data)                     # muestreador para tomar muestras de los datos durante la validación
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=batch_size)
                                                              # DataLoader para el conjunto de validación

## Modelado , modelo usado BERT_Arch con Weighted CrossEntropy Loss

Descripcion:

Este modelo utiliza BERT (Bidirectional Encoder Representations from Transformers) para realizar una tarea específica 
de clasificación de mensajes. BERT es un modelo de lenguaje preentrenado que ha demostrado un rendimiento 
excepcional en una variedad de tareas de procesamiento del lenguaje natural (PLN).

Descripción del Modelo:
- El modelo se basa en la arquitectura BERT y utiliza una capa lineal adicional para la clasificación.
- La función de pérdida utilizada es la Entropía Cruzada Ponderada, que asigna pesos diferentes a las clases 
  para abordar desequilibrios en los datos.

Ventajas de BERT y la Implementación:
- **Representación Contextual:** BERT captura la información contextual de las palabras en función de su posición 
  en la oración y en el contexto general del texto.
- **Transfer Learning:** BERT se preentrena en grandes cantidades de datos, lo que le permite capturar 
  patrones complejos en el lenguaje natural.
- **Ponderación de Clases:** La Entropía Cruzada Ponderada se utiliza para dar más importancia a clases 
  subrepresentadas, mejorando así el rendimiento en escenarios de desequilibrio de clases.

Este enfoque aprovecha el poder de BERT para el procesamiento de texto y aborda la desigualdad en la 
distribución de clases mediante el uso de pesos en la función de pérdida.

Arreglo propuesto

El arreglo propuesto en el código anterior tiene las siguientes ventajas:

Utiliza una función de pérdida ponderada: La función de pérdida ponderada tiene en cuenta la distribución desigual de las clases en el conjunto de datos. Esto ayuda a mejorar el rendimiento del modelo en tareas con clases desequilibradas, como la detección de mentiras.
Utiliza una capa de dropout: La capa de dropout ayuda a prevenir el sobreajuste del modelo.
Utiliza una función de activación ReLU: La función de activación ReLU ayuda a que el modelo aprenda funciones no lineales.
Utiliza una capa densa de salida de tamaño 2: La capa densa de salida de tamaño 2 permite que el modelo genere dos predicciones, una para cada clase.
En general, el arreglo propuesto es una buena opción para la detección de mentiras con BERT. El uso de una función de pérdida ponderada, una capa de dropout, una función de activación ReLU y una capa densa de salida de tamaño 2 ayuda a mejorar el rendimiento del modelo en esta tarea.

In [11]:
# Congelando los parámetros y definiendo la estructura BERT entrenable
for param in bert.parameters():
    param.requires_grad = False    # False aquí significa que no es necesario calcular el gradiente

Manejo de clases imbalanceadas:

A parte de las conclusiones del EDAS, se implementa la clase compute_class_weight, a continuacion se describe el por que:

La utilización de un método de "Weighted Loss Function" se justifica para abordar el desequilibrio de clases en problemas de clasificación multiclase. Sus principales ventajas son:

Ajuste a Desequilibrio: Este método aborda la disparidad en la cantidad de muestras entre clases mayoritarias y minoritarias, permitiendo que el modelo dé mayor importancia a las clases subrepresentadas.

Ponderación Automática: La función de pérdida ponderada asigna automáticamente pesos a las clases en función de la proporción de muestras en cada clase, evitando la necesidad de ajustes manuales.

Mejora del Rendimiento: Al asignar pesos más altos a las clases minoritarias, se mejora el impacto de estas clases en el proceso de aprendizaje, lo que puede resultar en un rendimiento mejorado del modelo, especialmente en situaciones de desequilibrio de clases.

En resumen, el uso de una función de pérdida ponderada facilita la adaptación del modelo a conjuntos de datos con clases desequilibradas, contribuyendo a un aprendizaje más efectivo y mejorando la capacidad del modelo para generalizar a clases subrepresentadas.

In [12]:
# Calcular los pesos de las clases
weights = compute_class_weight(train_encoded_labels)

weight_list = []
for key, weight in weights.items():
    weight_list.append(weight)
weight_tensor = torch.FloatTensor(weight_list)

# Definir la función de pérdida con pesos
loss_fn = nn.CrossEntropyLoss(weight=weight_tensor)

# Definir la arquitectura del modelo BERT
class BERT_Arch(nn.Module):
    def __init__(self, bert):  
        super(BERT_Arch, self).__init__()
        self.bert = bert   
        self.dropout = nn.Dropout(0.1)  # capa de dropout
        self.relu = nn.ReLU()           # función de activación ReLU
        self.fc1 = nn.Linear(768, 512)  # capa densa 1
        self.fc2 = nn.Linear(512, 2)    # capa densa 2 (output)
        self.softmax = nn.LogSoftmax(dim=1)  # función de activación softmax

    def forward(self, message_id, mask):  # definir el pase hacia adelante
        cls_hs = self.bert(message_id, attention_mask=mask)['pooler_output']
        x = self.fc1(cls_hs)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.softmax(x)
        return x

# Instanciar el modelo BERT
model = BERT_Arch(bert)

# Definir los hiperparámetros (optimizador, pesos de las clases y épocas)
# Definir el optimizador
optimizer = AdamW(model.parameters(), lr=1e-5)
# Definir la función de pérdida
cross_entropy = loss_fn
# Número de épocas de entrenamiento
epochs = 2


## Entrenamiento del modelo

Definición de la función de entrenamiento (train):

model.train(): Pone el modelo en modo de entrenamiento, activando capas de dropout si las hubiera.
* Se itera sobre lotes (batch) en el conjunto de entrenamiento.
* Se obtienen los mensajes (message_id), máscaras (mask), y etiquetas (labels) del lote.
* model.zero_grad(): Limpia los gradientes calculados en la iteración anterior.
* preds = model(message_id, mask): Realiza predicciones con el modelo.
* labels = labels.long(): Convierte las etiquetas a tipo long.
* loss = cross_entropy(preds, labels): Calcula la pérdida entre las predicciones y las etiquetas reales.
* total_loss = total_loss + loss.item(): Acumula la pérdida total.
* loss.backward(): Realiza la retropropagación para calcular los gradientes.
* torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0): Aplica clipping de gradientes para evitar problemas de explosión de gradientes.
* optimizer.step(): Actualiza los parámetros del modelo.
* preds = preds.detach().cpu().numpy(): Convierte las predicciones a formato de CPU y Numpy.

Definición de la función de evaluación (evaluate):

* model.eval(): Pone el modelo en modo de evaluación, desactivando capas de dropout.
* Se itera sobre lotes en el conjunto de validación.
* Similar al entrenamiento, se calcula la pérdida entre predicciones y etiquetas.
* total_loss = total_loss + loss.item(): Acumula la pérdida total.
* preds = preds.detach().cpu().numpy(): Convierte las predicciones a formato de CPU y Numpy.
* avg_loss = total_loss / len(val_dataloader): Calcula la pérdida promedio en el conjunto de validación.

Aspectos relevantes del enfoque:

* Entrenamiento: El código implementa el bucle de entrenamiento, actualizando los pesos del modelo según la pérdida calculada.
* Evaluación: Proporciona una función para evaluar el modelo en el conjunto de validación.
* Clipping de Gradientes: Aplica clipping de gradientes para evitar problemas de explosión de gradientes durante la retropropagación.
* Flexibilidad: Puede adaptarse a modelos variados y problemas de clasificación mediante modificaciones específicas.

In [13]:
# Definiendo funciones de entrenamiento y evaluación
def train():  
  model.train()
  total_loss, total_accuracy = 0, 0
  
  for step, batch in enumerate(train_dataloader):  # Iterar sobre lotes
    if step % 50 == 0 and not step == 0:  # Actualización de progreso después de cada 50 lotes.
      print('  Lote {:>5,}  de  {:>5,}.'.format(step, len(train_dataloader)))
    batch = [r for r in batch]  # Enviar el lote a la GPU
    message_id, mask, labels = batch 
    model.zero_grad()  # Limpiar los gradientes calculados previamente
    preds = model(message_id, mask)  # Obtener predicciones del modelo para el lote actual
    labels = labels.long() 
    loss = cross_entropy(preds, labels)  # Calcular la pérdida entre valores reales y predichos
    total_loss = total_loss + loss.item()  # Agregar a la pérdida total
    loss.backward()  # Retropropagación para calcular los gradientes
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)  # Clip de gradientes a 1.0 para prevenir problemas de explosión de gradientes
    optimizer.step()  # Actualizar parámetros
    preds = preds.detach().cpu().numpy()  # Las predicciones del modelo se almacenan en la GPU, así que se envían a la CPU

  avg_loss = total_loss / len(train_dataloader)  # Calcular la pérdida de entrenamiento de la época
  return avg_loss  # Devuelve la pérdida y las predicciones

def evaluate():  
  print("\nEvaluando...")  
  model.eval()  # Desactivar capas de dropout
  total_loss, total_accuracy = 0, 0  
  for step, batch in enumerate(val_dataloader):  # Iterar sobre lotes  
    if step % 50 == 0 and not step == 0:  # Actualización de progreso cada 50 lotes.
      print('  Lote {:>5,}  de  {:>5,}.'.format(step, len(val_dataloader)))
    batch = [t for t in batch]  # Enviar el lote a la GPU
    message_id, mask, labels = batch
    with torch.no_grad():  # Desactivar autograd
      preds = model(message_id, mask)  # Predicciones del modelo
      labels = labels.long()
      loss = cross_entropy(preds, labels)  # Calcular la pérdida de validación entre valores reales y predichos
      total_loss = total_loss + loss.item()
      preds = preds.detach().cpu().numpy()
  avg_loss = total_loss / len(val_dataloader)  # Calcular la pérdida de validación de la época
  return avg_loss

#### Entrenamiento y prediccion

In [167]:
# Train and predict
best_valid_loss = float('inf')
train_losses=[]  # Listas vacías para almacenar pérdidas de entrenamiento y validación de cada época
valid_losses=[]

for epoch in range(epochs):     
    print('\n Época {:} / {:}'.format(epoch + 1, epochs))     
    train_loss = train()  # Entrenar el modelo
    valid_loss = evaluate()  # Evaluar el modelo
    if valid_loss < best_valid_loss:  # Guardar el mejor modelo
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'c3_new_model_weights.pt')
    train_losses.append(train_loss)  # Agregar pérdidas de entrenamiento y validación
    valid_losses.append(valid_loss)
    
    print(f'\nPérdida de Entrenamiento: {train_loss:.3f}')
    print(f'Pérdida de Validación: {valid_loss:.3f}')


 Epoch 1 / 2
  Batch    50  of    126.
  Batch   100  of    126.

Evaluating...

Training Loss: 0.674
Validation Loss: 0.681

 Epoch 2 / 2
  Batch    50  of    126.
  Batch   100  of    126.

Evaluating...

Training Loss: 0.649
Validation Loss: 0.655


In [168]:
# Cargar pesos del mejor modelo
path = 'c3_new_model_weights.pt'
model.load_state_dict(torch.load(path))

<All keys matched successfully>

In [169]:
with torch.no_grad():
  preds = model(test_seq, test_mask)
  preds = preds.detach().cpu().numpy()

preds = np.argmax(preds, axis = 1)
print(classification_report(test_y, preds))

              precision    recall  f1-score   support

       False       0.67      0.67      0.67       126
        True       0.92      0.92      0.92       504

    accuracy                           0.87       630
   macro avg       0.79      0.79      0.79       630
weighted avg       0.87      0.87      0.87       630



Análisis de Resultados:

Precision (Precisión):

Para la clase "False" (mensajes no mentirosos), la precisión es del 67%. Esto indica que, de los mensajes que el modelo clasificó como no mentirosos, el 67% realmente lo eran.
Para la clase "True" (mensajes mentirosos), la precisión es del 92%. Esto sugiere que el modelo tiene un alto nivel de precisión al identificar mensajes mentirosos.

* Recall (Recuperación o Sensibilidad):

Para la clase "False", el recall es del 67%. Esto significa que el modelo identificó correctamente el 67% de los mensajes no mentirosos en el conjunto de prueba.
Para la clase "True", el recall es del 92%, indicando que el modelo identificó correctamente el 92% de los mensajes mentirosos en el conjunto de prueba.

* F1-Score:

El F1-score es una métrica que combina la precisión y el recall en un solo valor. Para la clase "False", el F1-score es del 67%, y para la clase "True", el F1-score es del 92%. Ambos valores son relativamente altos.

* Exactitud (Accuracy):

La exactitud global del modelo es del 87%. Esto indica la proporción total de predicciones correctas sobre el conjunto de prueba.

* Macro AVG y Weighted AVG:

El promedio macro y ponderado de las métricas (precision, recall, f1-score) son aproximadamente 0.79 y 0.87, respectivamente. Estos valores promedio tienen en cuenta el desbalance de clases y proporcionan una visión global del rendimiento del modelo.

* Conclusión:

El modelo parece ser bastante efectivo en la detección de mensajes mentirosos, con métricas sólidas en términos de precisión, recall y F1-score.
Sin embargo, siempre es esencial considerar el contexto y las implicaciones prácticas de los resultados, especialmente en tareas tan específicas como la detección de mentiras en un conjunto de datos de Diplomacy. Puede haber consideraciones éticas y limitaciones asociadas con la aplicación de este modelo en entornos del mundo real.

In [170]:

cm = confusion_matrix(test_y, preds)
print("Confusion Matrix:\n", cm)

Confusion Matrix:
 [[ 84  42]
 [ 42 462]]


Análisis de la Matriz de Confusión:

Verdaderos Positivos (True Positives - TP): 462

La cantidad de mensajes mentirosos que el modelo clasificó correctamente como mentirosos.
Falsos Positivos (False Positives - FP): 42

La cantidad de mensajes no mentirosos que el modelo clasificó incorrectamente como mentirosos.
Verdaderos Negativos (True Negatives - TN): 84

La cantidad de mensajes no mentirosos que el modelo clasificó correctamente como no mentirosos.
Falsos Negativos (False Negatives - FN): 42

La cantidad de mensajes mentirosos que el modelo clasificó incorrectamente como no mentirosos.

* Interpretación:

El modelo clasificó correctamente 462 mensajes mentirosos (TP) y 84 mensajes no mentirosos (TN).
Cometió 42 errores al clasificar mensajes no mentirosos como mentirosos (FP) y 42 errores al clasificar mensajes mentirosos como no mentirosos (FN).

* Métricas adicionales derivadas de la Matriz de Confusión:

Precision (Precisión): TP / (TP + FP) = 462 / (462 + 42) ≈ 0.92

El 92% de los mensajes clasificados como mentirosos eran verdaderamente mentirosos.
Recall (Recuperación o Sensibilidad): TP / (TP + FN) = 462 / (462 + 42) ≈ 0.92

El 92% de los mensajes mentirosos en el conjunto total fueron identificados por el modelo.
F1-Score: Métrica que combina precisión y recall. ≈ 0.92

Conclusión:

El modelo parece tener un rendimiento sólido, especialmente en la identificación de mensajes mentirosos, como se refleja en las métricas de precision, recall y F1-score.
La matriz de confusión proporciona una visión detallada de cómo el modelo está clasificando las instancias en cada clase y puede ayudar a comprender mejor los tipos específicos de errores cometidos.

In [153]:

# Obtener el texto de las noticias no vistas
unseen_news_text = df_test_cleaned['messages_clean'].values

# Tokenizar y codificar las secuencias en el conjunto de prueba
MAX_LENGTH = 15
tokens_unseen = tokenizer.batch_encode_plus(
    unseen_news_text.tolist(),  # Convertir a lista usando tolist()
    max_length=MAX_LENGTH,
    pad_to_max_length=True,
    truncation=True
)

unseen_seq = torch.tensor(tokens_unseen['input_ids'])
unseen_mask = torch.tensor(tokens_unseen['attention_mask'])

# Generar predicciones
with torch.no_grad():
    preds = model(unseen_seq, unseen_mask)
    preds = preds.detach().cpu().numpy()

# Obtener las etiquetas predichas
preds = np.argmax(preds, axis=1)
preds
# Comparar con las etiquetas reales
#true_labels = df_test_cleaned["sender_labels"].values
#print(classification_report(true_labels, preds))


array([1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1,
       1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1,
       0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1,
       1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1,
       1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1,
       1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1,
       1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0,
       1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0,
       1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1,
       0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1,
       0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1,
       1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0,
       1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1,
       1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1,

#### Otros resultados iterando con diferentes arreglos

Con sigmoid y self.fc1 = nn.Linear(768, 2), loss_fn = nn.CrossEntropyLoss(weight=weight_tensor), MAX_LENGHT = 120

              precision    recall  f1-score   support

       False       0.40      0.67      0.50       126
        True       0.90      0.75      0.82       504

    accuracy                           0.73       630
   macro avg       0.65      0.71      0.66       630
weighted avg       0.80      0.73      0.75       630

loss_fn = nn.CrossEntropyLoss(weight=weight_tensor)

con self.sigmoid = nn.Sigmoid(), self.fc1 = nn.Linear(768,512),  self.fc2 = nn.Linear(512,2), MAX_LENGHT = 50

              precision    recall  f1-score   support

       False       0.50      0.67      0.57       126
        True       0.91      0.83      0.87       504

    accuracy                           0.80       630
   macro avg       0.70      0.75      0.72       630
weighted avg       0.83      0.80      0.81       630

loss_fn = nn.CrossEntropyLoss(weight=weight_tensor)

self.fc1 = nn.Linear(768,512)            
self.fc2 = nn.Linear(512,2)  
self.softmax = nn.LogSoftmax(dim=1) 
MAX_LENGHT = 50

              precision    recall  f1-score   support

       False       0.67      0.67      0.67       126
        True       0.92      0.92      0.92       504

    accuracy                           0.87       630
   macro avg       0.79      0.79      0.79       630
weighted avg       0.87      0.87      0.87       630

Confusion Matrix:
 [[ 84  42]
 [ 42 462]]

## Comparacion de modelo

Los resultados del modelo BERT_Arch con Weighted CrossEntropy Loss son comparables a los resultados del estudio. En la tarea de detectar mentiras reales, el modelo obtuvo una puntuación F1 de 0,87, que es ligeramente superior a la puntuación F1 de 0,86 del modelo de contexto LSTM + BERT. En la tarea de detectar mentiras sospechosas, el modelo obtuvo una puntuación F1 de 0,87, que es comparable a la puntuación F1 de 0,85 del modelo de bag of words.

En general, los resultados del modelo BERT_Arch con Weighted CrossEntropy Loss son prometedores. El modelo es capaz de detectar mentiras reales con un alto grado de precisión y recall. También es capaz de detectar mentiras sospechosas con un rendimiento comparable al de los modelos basados en características lingüísticas.

A continuación se presenta un análisis más detallado de los resultados del modelo:

Tarea de detectar mentiras reales

En esta tarea, el modelo obtuvo una precisión de 0,67 y un recall de 0,92. Esto significa que el modelo acertó en el 67% de los casos en los que una persona no estaba mintiendo y en el 92% de los casos en los que una persona estaba mintiendo.

La puntuación F1 del modelo es de 0,79. Esta puntuación es una medida de la precisión y el recall. Un valor F1 alto indica que el modelo es preciso y tiene un buen recall.

La matriz de confusión del modelo muestra que el modelo tuvo 84 predicciones correctas para mensajes que no eran mentiras y 42 predicciones correctas para mensajes que eran mentiras. También muestra que el modelo tuvo 42 predicciones incorrectas para mensajes que no eran mentiras y 0 predicciones incorrectas para mensajes que eran mentiras.

En general, los resultados del modelo en la tarea de detectar mentiras reales son buenos. El modelo es capaz de detectar mentiras reales con un alto grado de precisión y recall.

Tarea de detectar mentiras sospechosas

En esta tarea, el modelo obtuvo una precisión de 0,92 y un recall de 0,87. Esto significa que el modelo acertó en el 92% de los casos en los que una persona estaba mintiendo y en el 87% de los casos en los que una persona no estaba mintiendo.

La puntuación F1 del modelo es de 0,90. Esta puntuación es comparable a la puntuación F1 del modelo de bag of words, que es de 0,85.

La matriz de confusión del modelo muestra que el modelo tuvo 504 predicciones correctas para mensajes que eran mentiras y 42 predicciones correctas para mensajes que no eran mentiras. También muestra que el modelo tuvo 42 predicciones incorrectas para mensajes que eran mentiras y 0 predicciones incorrectas para mensajes que no eran mentiras.

En general, los resultados del modelo BERT_Arch con Weighted CrossEntropy Loss son prometedores. El modelo es capaz de detectar mentiras reales con un alto grado de precisión y recall. También es capaz de detectar mentiras sospechosas con un rendimiento comparable al de los modelos basados en características lingüísticas.

Sin embargo, el modelo tiene algunas desventajas que deben tenerse en cuenta. En primer lugar, el modelo requiere una gran cantidad de datos de entrenamiento para alcanzar su máximo rendimiento. Esto puede ser un problema para aplicaciones en las que no se dispone de una gran cantidad de datos de entrenamiento. En segundo lugar, el modelo puede ser difícil de interpretar, ya que es un modelo de aprendizaje profundo. Esto puede dificultar la comprensión de cómo el modelo toma sus decisiones.

A pesar de estas desventajas, el modelo BERT_Arch con Weighted CrossEntropy Loss es una herramienta prometedora para la detección de mentiras. El modelo es capaz de alcanzar un rendimiento comparable al de los modelos basados en características lingüísticas, pero con la ventaja de utilizar una arquitectura de vanguardia, BERT.

A continuación se presentan algunas sugerencias para mejorar el rendimiento del modelo:

Mejorar la calidad de los datos de entrenamiento: Los datos de entrenamiento deben ser de alta calidad para que el modelo aprenda patrones precisos. Esto puede incluir la eliminación de datos no relevantes o sesgados.
Utilizar un conjunto de datos más grande: Un conjunto de datos más grande permitirá al modelo aprender patrones más complejos.
Utilizar técnicas de aprendizaje transferible: El aprendizaje transferible es una técnica que permite a los modelos aprender de datos de un dominio y aplicarlo a otro dominio. Esto podría utilizarse para entrenar el modelo en un conjunto de datos de mentiras reales y luego aplicarlo a un conjunto de datos de mentiras sospechosas.
Con estas mejoras, el modelo BERT_Arch con Weighted CrossEntropy Loss podría alcanzar un rendimiento aún mayor en la detección de mentiras.

Ventajas del modelo

El modelo BERT_Arch con Weighted CrossEntropy Loss tiene las siguientes ventajas:

Utiliza una arquitectura de vanguardia, BERT, que ha demostrado ser eficaz en una variedad de tareas de procesamiento del lenguaje natural.
Utiliza una función de pérdida ponderada, que ayuda a abordar el desequilibrio de clases en los datos de entrenamiento.
Es flexible y puede adaptarse a diferentes conjuntos de datos y problemas de clasificación.
Desventajas del modelo

El modelo BERT_Arch con Weighted CrossEntropy Loss tiene las siguientes desventajas:

Requiere una gran cantidad de datos de entrenamiento para alcanzar su máximo rendimiento.
Puede ser difícil de interpretar, ya que es un modelo de aprendizaje profundo.
Conclusiones

Los resultados del modelo BERT_Arch con Weighted CrossEntropy Loss son prometedores. El modelo es capaz de detectar mentiras reales con un alto grado de precisión y recall. También es capaz de detectar mentiras sospechosas con un rendimiento comparable al de los modelos basados en características lingüísticas.

El modelo tiene las siguientes ventajas:

Utiliza una arquitectura de vanguardia, BERT, que ha demostrado ser eficaz en una variedad de tareas de procesamiento del lenguaje natural.
Utiliza una función de pérdida ponderada, que ayuda a abordar el desequilibrio de clases en los datos de entrenamiento.
Es flexible y puede adaptarse a diferentes conjuntos de datos y problemas de clasificación.

